In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings 

from sklearn.ensemble import IsolationForest
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report


warnings.filterwarnings(action='ignore')

In [2]:
train = pd.read_csv("./mod_Data/train_remove_ver1.csv")
val_x = pd.read_csv("./mod_Data/val_x_remove_ver1.csv")
val_y = pd.read_csv("./mod_Data/val_y_remove_ver1.csv")
test_x = pd.read_csv("./mod_Data/test_x_remove_ver1.csv")

In [67]:
train = pd.read_csv("./Data/train.csv")
val_df = pd.read_csv("./Data/val.csv")
test_x = pd.read_csv("./Data/test.csv")

In [57]:
val_df.head()
val_df.drop('ID',axis=1,inplace=True)
val_x = val_df.drop(['Class','V29','V30'],axis=1)
val_y = val_df['Class']

In [58]:
train_x = train.drop(['ID','V29','V30'],axis=1)

In [70]:
train_x

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,...,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752
1,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,...,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458
2,-0.425966,0.960523,1.141109,-0.168252,0.420987,-0.029728,0.476201,0.260314,-0.568671,-0.371407,...,-0.033194,0.084968,-0.208254,-0.559825,-0.026398,-0.371427,-0.232794,0.105915,0.253844,0.081080
3,-0.644269,1.417964,1.074380,-0.492199,0.948934,0.428118,1.120631,-3.807864,0.615375,1.249376,...,0.324505,-0.156742,1.943465,-1.015455,0.057504,-0.649709,-0.415267,-0.051634,-1.206921,-1.085339
4,-0.894286,0.286157,-0.113192,-0.271526,2.669599,3.721818,0.370145,0.851084,-0.392048,-0.410430,...,0.570328,0.052736,-0.073425,-0.268092,-0.204233,1.011592,0.373205,-0.384157,0.011747,0.142404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
113837,-12.516732,10.187818,-8.476671,-2.510473,-4.586669,-1.394465,-3.632516,5.498583,4.893089,8.655320,...,-0.571605,3.490065,-0.944759,-1.565026,0.890675,-1.253276,1.786717,0.320763,2.090712,1.232864
113838,1.884849,-0.143540,-0.999943,1.506772,-0.035300,-0.613638,0.190241,-0.249058,0.666458,0.120908,...,-0.283278,-0.153997,0.144008,0.634646,-0.042114,-0.053206,0.316403,-0.461441,0.018265,-0.041068
113839,-0.241923,0.712247,0.399806,-0.463406,0.244531,-1.343668,0.929369,-0.206210,0.106234,-0.284708,...,-0.290491,-0.139512,-0.228876,-0.514376,0.279598,0.371441,-0.559238,0.113144,0.131507,0.081265
113840,0.120316,0.931005,-0.546012,-0.745097,1.130314,-0.235973,0.812722,0.115093,-0.204064,-0.657422,...,0.128904,0.000676,-0.314205,-0.808520,0.050343,0.102800,-0.435870,0.124079,0.217940,0.068803


In [59]:
# 머신러닝 모듈 사용하기
# sklearn의 preprocessing기능을 사용하면 최대값과 최소값을 비교하면서 정규화가 진행

from sklearn import preprocessing
# 정규화를 진행할 컬럼 설정
col = list(train_x.columns[:])

x = train_x[col].values

#최대값과 최소값의 값 가져오기
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler
# 정규화 시킬 최종 값은 비율로 계산되기 때문에 float설정
x_scaled = min_max_scaler.fit_transform(x.astype(float))
x_scaled
# 정규화된 범죄건수가 있는 새로운 데이터프레임 생성


array([[0.93521702, 0.75783971, 0.93410292, ..., 0.34500092, 0.46787594,
        0.21990117],
       [0.94187802, 0.77010241, 0.93464285, ..., 0.32835004, 0.4734903 ,
        0.2226898 ],
       [0.95105714, 0.78226755, 0.91685223, ..., 0.3942529 , 0.48257791,
        0.22314125],
       ...,
       [0.95418381, 0.77963144, 0.89662125, ..., 0.39570619, 0.47676092,
        0.2231455 ],
       [0.9603378 , 0.78195414, 0.87080886, ..., 0.39790423, 0.4808707 ,
        0.2228588 ],
       [0.75644829, 0.87900761, 0.61730826, ..., 0.42322368, 0.51537747,
        0.24022714]])

In [60]:
x_scaled[0]

array([0.93521702, 0.75783971, 0.93410292, 0.27070588, 0.54119475,
       0.55834526, 0.49436227, 0.72150286, 0.50024906, 0.61937519,
       0.35079379, 0.81409451, 0.54222436, 0.66794206, 0.65395661,
       0.48225951, 0.73344762, 0.65929885, 0.27683568, 0.52601252,
       0.46047565, 0.55995682, 0.67893917, 0.31186578, 0.5595152 ,
       0.34500092, 0.46787594, 0.21990117])

In [61]:
train_x_norm = pd.DataFrame(x_scaled, columns=col, index=train_x.index)
train_x_norm

train_x_norm.head()

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
0,0.935217,0.757840,0.934103,0.270706,0.541195,0.558345,0.494362,0.721503,0.500249,0.619375,...,0.276836,0.526013,0.460476,0.559957,0.678939,0.311866,0.559515,0.345001,0.467876,0.219901
1,0.941878,0.770102,0.934643,0.214438,0.547600,0.547297,0.487889,0.723332,0.505606,0.612767,...,0.383516,0.512500,0.453344,0.515525,0.662607,0.240850,0.614245,0.328350,0.473490,0.222690
2,0.951057,0.782268,0.916852,0.245899,0.553205,0.521798,0.490678,0.721681,0.539951,0.604802,...,0.507836,0.517901,0.451343,0.482764,0.665042,0.358283,0.564839,0.394253,0.482578,0.223141
3,0.947348,0.787124,0.915031,0.231235,0.560066,0.530941,0.498209,0.664343,0.589645,0.645592,...,0.544911,0.513446,0.494412,0.456349,0.666288,0.317645,0.554597,0.362583,0.413120,0.196306
4,0.943101,0.775107,0.882621,0.241224,0.582427,0.596712,0.489438,0.730007,0.547364,0.603820,...,0.570391,0.517307,0.454042,0.499677,0.662401,0.560251,0.598855,0.295739,0.471066,0.224552


In [62]:
# 가설 설정 : Train dataset도 Validation dataset과 동일한 비율로 사기거래가 발생 했을 것이다. -> model parameter : contamination=val_contamination(=0.001055) 적용
val_contamination  = 30 / 28462
model = IsolationForest(n_estimators=125, max_samples=len(train_x_norm), contamination=val_contamination, random_state=42, verbose=0)
model.fit(train_x_norm)

IsolationForest(contamination=0.0010540369615627855, max_samples=113842,
                n_estimators=125, random_state=42)

In [66]:
30 / 28462

0.0010540369615627855

In [63]:
def get_pred_label(model_pred):
    # IsolationForest 모델 출력 (1:정상, -1:불량(사기)) 이므로 (0:정상, 1:불량(사기))로 Label 변환
    model_pred = np.where(model_pred == 1, 0, model_pred)
    model_pred = np.where(model_pred == -1, 1, model_pred)
    return model_pred

In [64]:
col = list(val_x.columns[:])

x = val_x[col].values

#최대값과 최소값의 값 가져오기
min_max_scaler = preprocessing.MinMaxScaler()
min_max_scaler
# 정규화 시킬 최종 값은 비율로 계산되기 때문에 float설정
x_scaled = min_max_scaler.fit_transform(x.astype(float))
x_scaled

val_x_norm = pd.DataFrame(x_scaled, columns=col, index=val_x.index)
val_x_norm

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28
0,0.913867,0.719396,0.907517,0.282153,0.480733,0.562720,0.554520,0.680654,0.501486,0.635252,...,0.474907,0.435957,0.375847,0.540890,0.569616,0.374668,0.516025,0.362467,0.542617,0.350658
1,0.954608,0.704960,0.872176,0.417810,0.494456,0.631366,0.545051,0.692330,0.475360,0.667021,...,0.221627,0.424808,0.386527,0.611642,0.571247,0.222067,0.555864,0.385142,0.530194,0.346603
2,0.960340,0.709460,0.882799,0.446263,0.467851,0.534535,0.552431,0.672609,0.492316,0.667998,...,0.325433,0.426653,0.382896,0.575988,0.570615,0.541118,0.576899,0.352721,0.526435,0.347884
3,0.953498,0.693060,0.888426,0.326763,0.475233,0.629752,0.535731,0.694755,0.556831,0.639367,...,0.321183,0.426786,0.381486,0.589579,0.574523,0.280722,0.529418,0.423193,0.531260,0.346962
4,0.830336,0.746412,0.920286,0.154357,0.476530,0.542304,0.579137,0.631599,0.774244,0.822109,...,0.333343,0.489062,0.359308,0.571784,0.568646,0.513979,0.533849,0.440903,0.515730,0.301312
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
28457,0.907349,0.725133,0.868055,0.348965,0.481234,0.551452,0.561737,0.685828,0.480640,0.643668,...,0.530979,0.431323,0.388826,0.637336,0.568382,0.438913,0.523647,0.257375,0.545613,0.354415
28458,0.900517,0.709703,0.850779,0.270710,0.489491,0.544557,0.556383,0.689571,0.501823,0.609683,...,0.426502,0.431742,0.382689,0.554674,0.575542,0.525004,0.504670,0.399586,0.521043,0.344500
28459,0.910463,0.722728,0.872260,0.352196,0.482466,0.550039,0.564999,0.651923,0.519854,0.648162,...,0.504802,0.426364,0.405864,0.605004,0.568999,0.428226,0.522940,0.264520,0.556500,0.356753
28460,0.988342,0.695769,0.842372,0.308731,0.469669,0.545766,0.543473,0.675804,0.568850,0.653727,...,0.486136,0.425100,0.375270,0.535192,0.579067,0.378578,0.494755,0.385389,0.524941,0.344093


In [74]:

val_pred = model.predict(val_x_norm) # model prediction
val_pred = get_pred_label(val_pred)
val_score = f1_score(val_y, val_pred, average='macro')
print(f'Validation F1 Score : [{val_score}]')
print(classification_report(val_y, val_pred))

Validation F1 Score : [0.6514149625174386]
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     28432
           1       0.24      0.40      0.30        30

    accuracy                           1.00     28462
   macro avg       0.62      0.70      0.65     28462
weighted avg       1.00      1.00      1.00     28462

